In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet18
from torchvision.models.quantization import resnet18 as resnet18q
from torch.utils.mobile_optimizer import optimize_for_mobile
from torch.quantization import fuse_modules
from torch.utils.bundled_inputs import augment_model_with_bundled_inputs

## Look at TorchScript

In [5]:
# Load PyTorch model
model = resnet18(pretrained=True)
model.eval()

# Generate input image
example = torch.zeros(10, 3, 224, 224)

# Save model graph to TorchScript format
torchscript_model = torch.jit.trace(model, example)

In [11]:
print(torchscript_model.graph)

graph(%self.1 : __torch__.torchvision.models.resnet.___torch_mangle_92.ResNet,
      %input.1 : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu)):
  %1140 : __torch__.torch.nn.modules.linear.___torch_mangle_91.Linear = prim::GetAttr[name="fc"](%self.1)
  %1137 : __torch__.torch.nn.modules.pooling.___torch_mangle_90.AdaptiveAvgPool2d = prim::GetAttr[name="avgpool"](%self.1)
  %1136 : __torch__.torch.nn.modules.container.___torch_mangle_89.Sequential = prim::GetAttr[name="layer4"](%self.1)
  %1095 : __torch__.torch.nn.modules.container.___torch_mangle_73.Sequential = prim::GetAttr[name="layer3"](%self.1)
  %1054 : __torch__.torch.nn.modules.container.___torch_mangle_57.Sequential = prim::GetAttr[name="layer2"](%self.1)
  %1013 : __torch__.torch.nn.modules.container.___torch_mangle_41.Sequential = prim::GetAttr[name="layer1"](%self.1)
  %980 : __torch__.torch.nn.modules.pooling.___torch_mangle_28.MaxPool2d = prim::GetAttr[name="maxpool"](%self.1)
  %979

In [9]:
print(torchscript_model.code)

def forward(self,
    input: Tensor) -> Tensor:
  _0 = self.fc
  _1 = self.avgpool
  _2 = self.layer4
  _3 = self.layer3
  _4 = self.layer2
  _5 = self.layer1
  _6 = self.maxpool
  _7 = self.relu
  _8 = (self.bn1).forward((self.conv1).forward(input, ), )
  _9 = (_5).forward((_6).forward((_7).forward(_8, ), ), )
  _10 = (_2).forward((_3).forward((_4).forward(_9, ), ), )
  input0 = torch.flatten((_1).forward(_10, ), 1, -1)
  return (_0).forward(input0, )



### Why TorchScript?

- TorchScript code can be invoked in its own interpreter, which is basically a restricted Python interpreter. 

- This interpreter does not acquire the Global Interpreter Lock, and so many requests can be processed on the same instance simultaneously.

- This format allows us to save the whole model to disk and load it into another environment, such as in a server written in a language other than Python

- TorchScript gives us a representation in which we can do compiler optimizations on the code to provide more efficient execution

- TorchScript allows us to interface with many backend/device runtimes that require a broader view of the program than individual operators.

- We can see that invoking traced model produces the same results as the Python module

Inroduction to TorchScript: https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html


## Load original model

In [3]:
# Generate input image
example = torch.zeros(10, 3, 224, 224)

# Load PyTorch model
model = resnet18(pretrained=True)
model.eval()

# Fuse PyTorch model
layers_to_fuse = [["conv1", "bn1", "relu"]]
for i in range(1, 5):
    for j in range(2):
        for k in range(1, 3):
            layers_to_fuse += [[
                f'layer{i}.{j}.conv{k}', f'layer{i}.{j}.bn{k}'
            ]]

fuse_modules(model, layers_to_fuse, inplace=True)

# Save model graph to TorchScript format
torchscript_model = torch.jit.script(model)

# Optimize for mobile PyTorch operations that are supported by Android framework
# If operations are not supported, they remain unchanged
torchscript_model_optimized = optimize_for_mobile(torchscript_model)

# Save binary file with model on the computer (without input image example)
torchscript_model_optimized._save_for_lite_interpreter("resnet18_orig.ptl")

# Create a joint input consisting of model and input image
augment_model_with_bundled_inputs(torchscript_model_optimized, [(example, )])

# Save binary file with model on the computer (with input image example)
torchscript_model_optimized._save_for_lite_interpreter("resnet18_orig2.ptl")


## Load quantized model

In [4]:
# Load PyTorch model
model = resnet18q(pretrained=True, quantize=True)
model.eval()

# Generate input image
example = torch.zeros(10, 3, 224, 224)

# Save model graph to TorchScript format
torchscript_model = torch.jit.script(model)

# Optimize for mobile PyTorch operations that are supported by Android framework
# If operations are not supported, they remain unchanged
torchscript_model_optimized = optimize_for_mobile(torchscript_model)

# Save binary file with model on the computer (without input image example)
torchscript_model_optimized._save_for_lite_interpreter("resnet18_quan.ptl")

# Create a joint input consisting of model and input image
augment_model_with_bundled_inputs(torchscript_model_optimized, [(example, )])

# Save binary file with model on the computer (with input image example)
torchscript_model_optimized._save_for_lite_interpreter("resnet18_quan2.ptl")


/Users/fox/cloud/miniconda/envs/qals/lib/python3.7/site-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


## Load compressed model

In [5]:
def get_layer_by_name(model, mname: str):
    module = model
    mname_list = mname.split('.')
    for mname in mname_list:
        module = module._modules[mname]

    return module


def replace_layer_by_name(model, mname: str, layer):
    module = model
    mname_list = mname.split('.')
    for mname in mname_list[:-1]:
        module = module._modules[mname]

    module._modules[mname_list[-1]] = layer

In [6]:
import numpy as np

def get_cp3_decomposition(conv, cr):

    kernel_size = np.prod(conv.kernel_size)
    rank = int(cr * (kernel_size * conv.in_channels * conv.out_channels) /
               (kernel_size + conv.in_channels + conv.out_channels))

    cp3_conv = nn.Sequential(
        nn.Conv2d(in_channels=conv.in_channels,
                  out_channels=rank,
                  kernel_size=(1, 1),
                  bias=False),
        nn.Conv2d(in_channels=rank,
                  out_channels=rank,
                  kernel_size=conv.kernel_size,
                  groups=rank,
                  stride=conv.stride,
                  padding=conv.padding,
                  dilation=conv.dilation,
                  bias=False),
        nn.Conv2d(in_channels=rank,
                  out_channels=conv.out_channels,
                  kernel_size=(1, 1),
                  bias=conv.bias is not None))

    return cp3_conv

In [7]:
# Choose compression ratio
cr = 0.5

# Load PyTorch model
model = resnet18(pretrained=True)


# Select convolutional layers
conv_layers = [
    module_name for module_name, module in model.named_modules()
    if isinstance(module, nn.Conv2d) and "downsample" not in module_name
]

# Compress model
for layer_name in conv_layers:
    layer = get_layer_by_name(model, layer_name)
    cp3_decomposition = get_cp3_decomposition(layer, cr)
    replace_layer_by_name(model, layer_name, cp3_decomposition)

# Fuse model
layers_to_fuse = [["conv1.2", "bn1", "relu"]]
for i in range(1, 5):
    for j in range(2):
        for k in range(1, 3):
            layers_to_fuse += [[
                f'layer{i}.{j}.conv{k}.2', f'layer{i}.{j}.bn{k}'
            ]]

model.eval()
fuse_modules(model, layers_to_fuse, inplace=True)

# Save model
torchscript_model = torch.jit.script(model)
torchscript_model_optimized = optimize_for_mobile(torchscript_model)
torchscript_model_optimized._save_for_lite_interpreter("resnet18_comp.ptl")

## Load compressed quantized model

In [10]:
# Choose compression ratio
cr = 0.5

# Load PyTorch model
model = resnet18q(pretrained=True, quantize=False)

# Select convolutional layers
conv_layers = [
    module_name for module_name, module in model.named_modules()
    if isinstance(module, nn.Conv2d) and "downsample" not in module_name
]

# Compress model
for layer_name in conv_layers:
    layer = get_layer_by_name(model, layer_name)
    cp3_decomposition = get_cp3_decomposition(layer, cr)
    replace_layer_by_name(model, layer_name, cp3_decomposition)

# # Fuse model
# layers_to_fuse = [["conv1.2", "bn1", "relu"]]
# for i in range(1, 5):
#     for j in range(2):
#         for k in range(1, 3):
#             layers_to_fuse += [[
#                 f'layer{i}.{j}.conv{k}.2', f'layer{i}.{j}.bn{k}'
#             ]]

# model.eval()
# fuse_modules(model, layers_to_fuse, inplace=True)

# Quantize model
model.qconfig = torch.quantization.get_default_qconfig('qnnpack')
torch.quantization.prepare(model, inplace=True)
# Calibrate your model
# def calibrate(model, calibration_data):
#     # Your calibration code here
#     return
# calibrate(model, [])
torch.quantization.convert(model, inplace=True)

# Save model
torchscript_model = torch.jit.script(model)
torchscript_model_optimized = optimize_for_mobile(torchscript_model)
augment_model_with_bundled_inputs(torchscript_model_optimized, [(example, )])
torchscript_model_optimized._save_for_lite_interpreter(
    "resnet18_comp_quan.ptl")

In [11]:
 import os
 print('Size (MB):', os.path.getsize("resnet18_orig.ptl") / 1024**2)
 print('Size (MB):', os.path.getsize("resnet18_quan.ptl") / 1024**2)
 print('Size (MB):', os.path.getsize("resnet18_comp.ptl") / 1024**2)
 print('Size (MB):', os.path.getsize("resnet18_comp_quan.ptl") / 1024**2)

Size (MB): 89.17584037780762
Size (MB): 22.5590877532959
Size (MB): 47.225582122802734
Size (MB): 12.061223983764648


## Commands for ADB

In [ ]:
!mv resnet18*.plt app/src/main/assets/

In [ ]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile


class AnnotatedConvBnReLUModel(torch.nn.Module):

    def __init__(self):
        super(AnnotatedConvBnReLUModel, self).__init__()
        self.conv = torch.nn.Conv2d(3, 5, 3, bias=False).to(dtype=torch.float)
        # self.bn = torch.nn.BatchNorm2d(5).to(dtype=torch.float)
        self.relu = torch.nn.ReLU(inplace=True)
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):
        #         x = x.contiguous(memory_format=torch.channels_last)
        x = self.quant(x)
        x = self.conv(x)
        # x = self.bn(x)
        x = self.relu(x)
        x = self.dequant(x)
        return x


model = AnnotatedConvBnReLUModel()

torch.quantization.fuse_modules(model, [['conv', 'relu']], inplace=True)

model.qconfig = torch.quantization.get_default_qconfig('fbgemm') #'qnnpack') 'fbgemm'
torch.quantization.prepare(model, inplace=True)
# Calibrate your model
# def calibrate(model, calibration_data):
#     # Your calibration code here
#     return
# calibrate(model, [])
torch.quantization.convert(model, inplace=True)

AnnotatedConvBnReLUModel(
  (conv): ConvReLU2d(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ReLU(inplace=True)
  )
  (relu): Identity()
  (quant): QuantStub()
  (dequant): DeQuantStub()
)

In [21]:
torchscript_model = torch.jit.script(model)
torchscript_model_optimized = optimize_for_mobile(torchscript_model)
torchscript_model_optimized._save_for_lite_interpreter("model.ptl")

## Lo

In [2]:
import tensorly as tl 
from tensorly.decomposition import parafac
tl.set_backend("pytorch")
tl.__version__

'0.4.5'

In [3]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile


class ModelALS(torch.nn.Module):

    def __init__(self):
        super(ModelALS, self).__init__()

    def forward(self, A, b):

        x = torch.solve(A, b)

        return x

x = torch.randn(10, 11, 12)
Rcp = 5
examples = [(x, Rcp, i) for i in [100, 200, 300]]

model = ModelALS()

# Save model graph to TorchScript format
torchscript_model = torch.jit.script(model)

# Optimize for mobile PyTorch operations that are supported by Android framework
# If operations are not supported, they remain unchanged
torchscript_model_optimized = optimize_for_mobile(torchscript_model)

# Create a joint input consisting of model and input image
augment_model_with_bundled_inputs(torchscript_model_optimized, examples)

# Save binary file with model on the computer (with input image example)
torchscript_model_optimized._save_for_lite_interpreter("model2.ptl")

RuntimeError: 
undefined value parafac:
  File "<ipython-input-3-adc1a3bf8e63>", line 12
    def forward(self, X, rank, iters):
    
        weights, factors = parafac(X, rank, iters, normalize_factors=True)
                           ~~~~~~~ <--- HERE
    
        return weights, factors
